In [1]:
import os
import glob
import yaml

import numpy as np
import pandas as pd
import polars as pl
import pyranges as pr
import pyfaidx
import scipy
import torch
import tqdm
import anndata as ad
import pyranges
import plotnine as p9

from sklearn.metrics import average_precision_score, roc_auc_score

# Get variants

## Load EBI Susie finemapped variants

In [2]:
# Downloaded from https://github.com/eQTL-Catalogue/eQTL-Catalogue-resources/blob/master/data_tables/dataset_metadata.tsv
susie_datasets = pd.read_table('Data/Susie/dataset_metadata.tsv').query('quant_method == "ge" and study_label == "GTEx"')

In [3]:
# Downloaded from http://ftp.ebi.ac.uk/pub/databases/spot/eQTL
susie_df_list = []

for _,row in susie_datasets.iterrows():
    study_id = row['study_id']
    dataset_id = row['dataset_id']
    path = f'Data/Susie/{study_id}/{dataset_id}/{dataset_id}.credible_sets.tsv.gz'
    susie_df = pd.read_table(path, compression='gzip')
    susie_df = susie_df.query('pip > 0.9') # select high-pip variants
    susie_df['study_label'] = row['study_label']
    susie_df['study_id'] = row['study_id']
    susie_df['dataset_id'] = row['dataset_id']
    susie_df['tissue_label'] = row['tissue_label']
    susie_df['condition_label'] = row['condition_label']
    susie_df['sample_size'] = row['sample_size']
    susie_df['Chromosome'] = susie_df['variant'].apply(lambda x: x.split('_')[0])
    susie_df['Pos'] = susie_df['variant'].apply(lambda x: x.split('_')[1]).astype('int') - 1
    susie_df['Ref'] = susie_df['variant'].apply(lambda x: x.split('_')[2])
    susie_df['Alt'] = susie_df['variant'].apply(lambda x: x.split('_')[3])
    susie_df = susie_df.loc[(susie_df['Ref'].str.len() == 1) & (susie_df['Alt'].str.len() == 1)]
    susie_df_list.append(susie_df)
    
susie_df = pd.concat(susie_df_list).reset_index(drop=True)

In [4]:
susie_df = susie_df[['variant','gene_id','tissue_label','beta']]

## Load GTEx log(aFC)

In [5]:
# Downloaded from https://www.gtexportal.org/home/downloads/adult-gtex/qtl
gtex_df_list = []

for path in glob.glob('Data/Susie/GTEx_Analysis_v10_eQTL_updated/*.v10.eGenes.txt.gz'):
    #path = f'Data/Susie/GTEx_Analysis_v10_eQTL_updated/Adipose_Subcutaneous.v10.eGenes.txt.gz'
    gtex_df = pd.read_table(path, compression='gzip')[['variant_id','gene_id','slope','afc']]#.columns
    gtex_df['variant_id'] = gtex_df['variant_id'].str.replace('_b38','') 
    gtex_df['tissue'] = path.split('/')[-1].split('.')[0].lower()
    gtex_df['gene_id'] = gtex_df['gene_id'].apply(lambda x: x.split('.')[0])
    gtex_df_list.append(gtex_df.rename(columns={'tissue':'tissue_label_gtex','variant_id':'variant'}))

gtex_df = pd.concat(gtex_df_list).reset_index(drop=True)

## Merge

In [7]:
mapping = {
     'LCL':'cells_ebv-transformed_lymphocytes',
     'adipose':'adipose_subcutaneous',
     'adipose (visceral)':'adipose_visceral_omentum',
     'adrenal gland':'adrenal_gland',
     'artery (aorta)':'artery_aorta',
     'artery (coronary)':'artery_coronary',
     'artery (tibial)':'artery_tibial',
     'blood':'whole_blood',
     'brain (DLPFC)':'brain_frontal_cortex_ba9',
     'brain (amygdala)':'brain_amygdala',
     'brain (anterior cingulate cortex)':'brain_anterior_cingulate_cortex_ba24',
     'brain (caudate)':'brain_caudate_basal_ganglia',
     'brain (cerebellum)':'brain_cerebellum',
     'brain (cortex)':'brain_cortex',
     'brain (hippocampus)':'brain_hippocampus',
     'brain (hypothalamus)':'brain_hypothalamus',
     'brain (nucleus accumbens)':'brain_nucleus_accumbens_basal_ganglia',
     'brain (putamen)':'brain_putamen_basal_ganglia',
     'brain (spinal cord)':'brain_spinal_cord_cervical_c-1',
     'brain (substantia nigra)':'brain_substantia_nigra',
     'breast':'breast_mammary_tissue',
     'esophagus (gej)':'esophagus_gastroesophageal_junction',
     'esophagus (mucosa)':'esophagus_mucosa',
     'esophagus (muscularis)':'esophagus_muscularis',
     'fibroblast':'cells_cultured_fibroblasts',
     'heart (atrial appendage)':'heart_atrial_appendage',
     'heart (left ventricle)':'heart_left_ventricle',
     'kidney (cortex)':'kidney_cortex',
     'liver':'liver',
     'lung':'lung',
     'minor salivary gland':'minor_salivary_gland',
     'muscle':'muscle_skeletal',
     'ovary':'ovary',
     'pancreas':'pancreas',
     'pituitary':'pituitary',
     'prostate':'prostate',
     'sigmoid colon':'colon_sigmoid',
     'skin':'skin_sun_exposed_lower_leg',
     'skin (suprapubic)':'skin_not_sun_exposed_suprapubic',
     'small intestine':'small_intestine_terminal_ileum',
     'spleen':'spleen',
     'stomach':'stomach',
     'testis':'testis',
     'thyroid':'thyroid',
     'tibial nerve':'nerve_tibial',
     'transverse colon':'colon_transverse',
     'uterus':'uterus',
     'vagina':'vagina',
}

In [8]:
susie_df['tissue_label_gtex'] = susie_df['tissue_label'].apply(lambda x: mapping[x])

In [9]:
merged_df = gtex_df.merge(susie_df,on=['tissue_label_gtex','variant','gene_id'])

In [16]:
merged_df = merged_df.dropna(subset='afc')

In [17]:
print(scipy.stats.pearsonr(merged_df['beta'],merged_df['slope']))
print(scipy.stats.spearmanr(merged_df['beta'],merged_df['slope']))

PearsonRResult(statistic=np.float64(0.9896187093377892), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.9853893701892263), pvalue=np.float64(0.0))


In [18]:
print(scipy.stats.pearsonr(merged_df['beta'],merged_df['afc']))
print(scipy.stats.spearmanr(merged_df['beta'],merged_df['afc']))

PearsonRResult(statistic=np.float64(0.846221440020515), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.913184012927561), pvalue=np.float64(0.0))
